In [21]:
'''

@Author: Vighnesh Harish Bilgi
@Date: 2022-11-20
@Last Modified by: Vighnesh Harish Bilgi
@Last Modified time: 2022-11-20
@Title : To fetch tweets from Twitter API

'''

'\n\n@Author: Vighnesh Harish Bilgi\n@Date: 2022-11-20\n@Last Modified by: Vighnesh Harish Bilgi\n@Last Modified time: 2022-11-20\n@Title : To fetch tweets from Twitter API\n\n'